In [1]:
from pyomo.environ import *
import mpisppy.utils.sputils as sputils
import matplotlib.pyplot as plt
from matplotlib import rc

[    0.00] Initializing mpi-sppy


In [2]:
def build_model(yields):
    model = ConcreteModel()

    # Variables
    model.X = Var(["WHEAT", "CORN", "BEETS"], within=NonNegativeReals)
    model.Y = Var(["WHEAT", "CORN"], within=NonNegativeReals)
    model.W = Var(
        ["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"],
        within=NonNegativeReals,
    )

    # Objective function
    model.PLANTING_COST = 150 * model.X["WHEAT"] + 230 * model.X["CORN"] + 260 * model.X["BEETS"]
    model.PURCHASE_COST = 238 * model.Y["WHEAT"] + 210 * model.Y["CORN"]
    model.SALES_REVENUE = (
        170 * model.W["WHEAT"] + 150 * model.W["CORN"]
        + 36 * model.W["BEETS_FAVORABLE"] + 10 * model.W["BEETS_UNFAVORABLE"]
    )
    model.OBJ = Objective(
        expr=model.PLANTING_COST + model.PURCHASE_COST - model.SALES_REVENUE,
        sense=minimize
    )

    # Constraints
    model.CONSTR= ConstraintList()

    model.CONSTR.add(summation(model.X) <= 500)
    model.CONSTR.add(
        yields[0] * model.X["WHEAT"] + model.Y["WHEAT"] - model.W["WHEAT"] >= 200
    )
    model.CONSTR.add(
        yields[1] * model.X["CORN"] + model.Y["CORN"] - model.W["CORN"] >= 240
    )
    model.CONSTR.add(
        yields[2] * model.X["BEETS"] - model.W["BEETS_FAVORABLE"] - model.W["BEETS_UNFAVORABLE"] >= 0
    )
    model.W["BEETS_FAVORABLE"].setub(6000)

    return model

In [3]:
yields = [2.5, 3, 20]
model = build_model(yields)
solver = SolverFactory("gurobi")
solver.solve(model)

{'Problem': [{'Name': 'x1', 'Lower bound': -118600.0, 'Upper bound': -118600.0, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 9, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 9, 'Number of nonzeros': 12, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': 0, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': 0.0, 'Error rc': 0}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [4]:
print(f"{value(model.OBJ):.1f}")

-118600.0


In [5]:
import mpisppy.utils.sputils as sputils

def scenario_creator(scenario_name):
    if scenario_name == "good":
        yields = [3, 3.6, 24]
    elif scenario_name == "average":
        yields = [2.5, 3, 20]
    elif scenario_name == "bad":
        yields = [2, 2.4, 16]
    else:
        raise ValueError("Unrecognized scenario name")
    
    scenario_probabilities = {'good': 0.33, 'average': 0.33, 'bad':0.33} 
    model = build_model(yields)
    sputils.attach_root_node(model, model.PLANTING_COST, [model.X])
    model._mpisppy_probability = scenario_probabilities[scenario_name]
    # model.pprint()
    return model

In [6]:
from mpisppy.opt.ef import ExtensiveForm

options = {"solver": "gurobi"}
all_scenario_names = ["good", "average", "bad"]
ef = ExtensiveForm(options, all_scenario_names, scenario_creator)
results = ef.solve_extensive_form()

objval = ef.get_objective_value()
print(f"{objval:.2f}")

[    3.00] Initializing SPBase
-108390.00


In [37]:
output_dict = dict()

In [38]:
for name in all_scenario_names:
    model_vars = getattr(ef.ef, name).component_map(ctype=Var)
    vars_dict = {i: model_vars[i].extract_values() for i in model_vars.keys()}
    
    model_obj = getattr(ef.ef, name).component_map(ctype=Objective)
    obj_dict = {'objective': model_obj[i]() for i in model_obj.keys()}
    
    output_dict[name] = {**vars_dict, **obj_dict}

In [39]:
output_dict

{'good': {'X': {'WHEAT': 170.0, 'CORN': 80.0, 'BEETS': 250.0},
  'Y': {'WHEAT': 0.0, 'CORN': 0.0},
  'W': {'WHEAT': 310.0,
   'CORN': 48.0,
   'BEETS_FAVORABLE': 6000.0,
   'BEETS_UNFAVORABLE': 0.0},
  'objective': -167000.0},
 'average': {'X': {'WHEAT': 170.0, 'CORN': 80.0, 'BEETS': 250.0},
  'Y': {'WHEAT': 0.0, 'CORN': 0.0},
  'W': {'WHEAT': 225.0,
   'CORN': 0.0,
   'BEETS_FAVORABLE': 5000.0,
   'BEETS_UNFAVORABLE': 0.0},
  'objective': -109350.0},
 'bad': {'X': {'WHEAT': 170.0, 'CORN': 80.0, 'BEETS': 250.0},
  'Y': {'WHEAT': 0.0, 'CORN': 48.0},
  'W': {'WHEAT': 140.0,
   'CORN': 0.0,
   'BEETS_FAVORABLE': 4000.0,
   'BEETS_UNFAVORABLE': 0.0},
  'objective': -48820.0}}

In [7]:
# from mpisppy.utils.sputils import  create_EF
# 
# EF = create_EF(scenario_names=all_scenario_names, scenario_creator=scenario_creator, EF_name='Test EF')
# EF.write('Test.lp')

In [8]:
soln = ef.get_root_solution()
for (var_name, var_val) in soln.items():
    print(var_name, var_val)

X[BEETS] 250.0
X[CORN] 80.0
X[WHEAT] 170.0


In [9]:
results

{'Problem': [{'Name': 'x1', 'Lower bound': -108390.00000000003, 'Upper bound': -108390.00000000003, 'Number of objectives': 1, 'Number of constraints': 18, 'Number of variables': 27, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 27, 'Number of nonzeros': 48, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': 0, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': 0.0, 'Error rc': 0}], 'Solution': [OrderedDict({'number of solutions': 1, 'number of solutions displayed': 1}), {'Gap': 0.0, 'Status': 'optimal', 'Problem': {}, 'Objective': {'__default_objective__': {'Value': -108390.00000000003}}, 'Variable': {'x2': {'Value': 170.0}, 'x3': {'Value': 80.0}, 'x4': {'Value': 250.0}, 'x5': {'Value': 0.0}, '

In [10]:
# from mpisppy.opt.ph import PH
# 
# options = {
#     "solver_name": "gurobi",
#     "PHIterLimit": 5,
#     "defaultPHrho": 10,
#     "convthresh": 1e-7,
#     "verbose": False,
#     "display_progress": False,
#     "display_timing": False,
#     "iter0_solver_options": dict(),
#     "iterk_solver_options": dict(),
# }
# all_scenario_names = ["good", "average", "bad"]
# ph = PH(
#     options,
#     all_scenario_names,
#     scenario_creator,
# )

In [11]:
# ph.ph_main()

In [12]:
from mpisppy.opt.lshaped import LShapedMethod

all_scenario_names = ["good", "average", "bad"]
bounds = {name: -432000 for name in all_scenario_names}
options = {
    "root_solver": "gurobi",
    "sp_solver": "gurobi",
    "sp_solver_options" : {"threads" : 1},
    "valid_eta_lb": bounds,
    "max_iter": 10,
}

ls = LShapedMethod(options, all_scenario_names, scenario_creator)
result = ls.lshaped_algorithm()

variables = ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

[    3.04] Initializing SPBase


ImportError: BendersCutGenerator requires mpi4py.

In [13]:
result

NameError: name 'result' is not defined

In [14]:
ef.write_tree_solution('efsol_farmer')